# KerasNLP로 모델 업로드

**저자:** [Samaneh Saadat](https://github.com/SamanehSaadat/), [Matthew Watson](https://github.com/mattdangerw/)  
**생성일:** 2024/04/29  
**최종편집일:** 2024/04/29  
**설명:** 미세 조정된 KerasNLP 모델을 모델 허브에 업로드하는 방법에 대한 소개.

## 소개

머신 러닝 모델을 특정 작업에 맞게 파인 튜닝하면 인상적인 결과를 얻을 수 있습니다.
파인 튜닝된 모델을 모델 허브에 업로드하면 더 넓은 커뮤니티와 공유할 수 있으며,
이를 통해 다른 연구자들과 개발자들이 접근성을 높일 수 있습니다.
또한, 이를 통해 실세계 애플리케이션에 모델을 통합하는 과정이 간소화될 수 있습니다.

이 가이드에서는 파인 튜닝된 모델을 [Kaggle Models](https://www.kaggle.com/models)와
[Hugging Face Hub](https://huggingface.co/models)와 같은 인기 있는 모델 허브에 업로드하는 방법을 설명합니다.

## 셋업

우선 필요한 라이브러리를 설치하고 import 합니다.
이 가이드에서는 KerasNLP를 사용합니다.

In [2]:
# 이 노트북은 Keras nlp가 설치되어 있다는 가정 하에 진행됩니다.
# 이 노트북은 Keras 3이 설치되어 있다는 가정 하에 진행됩니다.
#
# !pip install -q --upgrade keras-nlp
# !pip install -q --upgrade keras  # Upgrade to Keras 3.

In [3]:
# !pip install -q --upgrade keras-nlp huggingface-hub kagglehub
!pip install -q --upgrade huggingface-hub kagglehub

In [4]:
import os

os.environ["KERAS_BACKEND"] = "jax"

import keras_nlp


## 데이터

이 가이드에서는 IMDB 리뷰 데이터셋을 사용합니다. `tensorflow_dataset`에서 데이터셋을 불러옵니다.

In [5]:
import tensorflow_datasets as tfds

imdb_train, imdb_test = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    as_supervised=True,
    batch_size=4,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO45AG2/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO45AG2/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO45AG2/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


가이드 실행 속도를 높이기 위해 트레이닝 샘플 중 일부만 사용합니다.
하지만, 더 높은 품질의 모델이 필요하다면, 더 많은 트레이닝 샘플을 사용하는 것이 좋습니다.

In [6]:
imdb_train = imdb_train.take(100)

## 작업 업로드

[`keras_nlp.models.Task`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/task#task-class)는 [`keras_nlp.models.Backbone`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/backbone#backbone-class)과 [`keras_nlp.models.Preprocessor`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/preprocessor#preprocessor-class)를 결합하여,
텍스트 문제에 대해 직접 트레이닝, 파인 튜닝 및 예측에 사용할 수 있는 모델을 생성합니다.
이 섹션에서는, `Task`를 생성하고, 파인 튜닝하여 모델 허브에 업로드하는 방법을 설명합니다.

### 모델 로드

베이스 모델을 기반으로 Causal LM을 구축하려면,
[`keras_nlp.models.CausalLM.from_preset`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/causal_lm#frompreset-method)를 호출하고,
미리 설정된 식별자(built-in preset identifier)를 전달하면 됩니다.

In [7]:
causal_lm = keras_nlp.models.CausalLM.from_preset("gpt2_base_en")

100%|██████████| 484/484 [00:00<00:00, 216kB/s]


100%|██████████| 475M/475M [00:26<00:00, 19.1MB/s]


100%|██████████| 448/448 [00:00<00:00, 138kB/s]


100%|██████████| 0.99M/0.99M [00:01<00:00, 750kB/s]


100%|██████████| 446k/446k [00:01<00:00, 407kB/s]


### 모델 미세 조정

모델을 불러온 후, `.fit()`을 호출하여 파인 튜닝할 수 있습니다.
여기에서는 IMDB 리뷰에 모델을 파인 튜닝하여 모델을 영화 도메인에 맞게 최적화합니다.

In [8]:
# Causal LM을 위해 레이블을 제거하고 리뷰 텍스트만 유지합니다.
imdb_train_reviews = imdb_train.map(lambda x, y: x)

# Causal LM을 파인 튜닝합니다.
causal_lm.fit(imdb_train_reviews)

100/100 ━━━━━━━━━━━━━━━━━━━━ 38s 260ms/step - loss: 1.0185 - sparse_categorical_accuracy: 0.3280


### 모델을 로컬에 저장하기

모델을 업로드하려면, 먼저 `save_to_preset`을 사용하여 모델을 로컬에 저장해야 합니다.

In [9]:
preset_dir = "./gpt2_imdb"
causal_lm.save_to_preset(preset_dir)

저장된 파일을 확인해 봅시다.

In [10]:
os.listdir(preset_dir)

['metadata.json',
 'tokenizer.json',
 'assets',
 'config.json',
 'task.json',
 'preprocessor.json',
 'model.weights.h5']

#### 로컬에 저장된 모델 불러오기

로컬에 저장된 프리셋 모델은 `from_preset`을 사용하여 불러올 수 있습니다.
저장된 그대로 불러올 수 있습니다.

In [11]:
causal_lm = keras_nlp.models.CausalLM.from_preset(preset_dir)

또한 [`keras_nlp.models.Backbone`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/backbone#backbone-class) 및 [`keras_nlp.models.Tokenizer`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/tokenizer#tokenizer-class) 객체를 프리셋 디렉토리에서 불러올 수 있습니다.
이 객체들은 위에서 사용한 `causal_lm.backbone` 및 `causal_lm.preprocessor.tokenizer`와 동일합니다.

In [12]:
backbone = keras_nlp.models.Backbone.from_preset(preset_dir)
tokenizer = keras_nlp.models.Tokenizer.from_preset(preset_dir)

### 모델 허브에 모델 업로드

프리셋을 디렉토리에 저장한 후,
이 디렉토리는 KerasNLP 라이브러리를 통해 Kaggle 또는 Hugging Face와 같은 모델 허브에 직접 업로드할 수 있습니다.
Kaggle에 모델을 업로드하려면 URI가 `kaggle://`로 시작해야 하고,
Hugging Face에 업로드하려면 `hf://`로 시작해야 합니다.

#### Kaggle에 업로드

Kaggle에 모델을 업로드하려면 먼저 Kaggle에 인증해야 합니다.
다음 중 하나의 방법을 사용할 수 있습니다:

1. 환경 변수 `KAGGLE_USERNAME`과 `KAGGLE_KEY`를 설정합니다.
2. 로컬에 `~/.kaggle/kaggle.json` 파일을 제공합니다.
3. `kagglehub.login()`을 호출합니다.

계속하기 전에 로그인 상태를 확인해 봅시다.

In [ ]:
import kagglehub

if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()


모델을 업로드하려면 `keras_nlp.upload_preset(uri, preset_dir)` API를 사용할 수 있으며,
Kaggle에 업로드할 경우 `uri`의 형식은 `kaggle://<KAGGLE_USERNAME>/<MODEL>/Keras/<VARIATION>`입니다.
`preset_dir`은 모델이 저장된 디렉토리입니다.

다음 명령을 실행하면 `preset_dir`에 저장된 모델을 Kaggle에 업로드할 수 있습니다:

In [ ]:
kaggle_username = kagglehub.whoami()["username"]
kaggle_uri = f"kaggle://{kaggle_username}/gpt2/keras/gpt2_imdb"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

#### Hugging Face에 업로드

Hugging Face에 모델을 업로드하려면 먼저 Hugging Face에 인증해야 합니다.
다음 중 하나의 방법을 사용할 수 있습니다:

1. 환경 변수 `HF_USERNAME`과 `HF_TOKEN`을 설정합니다.
2. `huggingface_hub.notebook_login()`을 호출합니다.

계속하기 전에 로그인 상태를 확인해 봅시다.

In [ ]:
import huggingface_hub

if "HF_USERNAME" not in os.environ or "HF_TOKEN" not in os.environ:
    huggingface_hub.notebook_login()

`keras_nlp.upload_preset(uri, preset_dir)`는 Hugging Face에 모델을 업로드할 때 사용할 수 있으며,
이 경우 `uri` 형식은 `hf://<HF_USERNAME>/<MODEL>`입니다.

다음 명령을 실행하면 `preset_dir`에 저장된 모델을 Hugging Face에 업로드할 수 있습니다:

In [ ]:
hf_username = huggingface_hub.whoami()["name"]
hf_uri = f"hf://{hf_username}/gpt2_imdb"
keras_nlp.upload_preset(hf_uri, preset_dir)


### 사용자가 업로드한 모델 불러오기

Kaggle에 모델이 업로드된 것을 확인한 후, `from_preset`을 호출하여 모델을 불러올 수 있습니다.

```python
causal_lm = keras_nlp.models.CausalLM.from_preset(
    f"kaggle://{kaggle_username}/gpt2/keras/gpt2_imdb"
)
```

Hugging Face에 업로드된 모델도 `from_preset`을 호출하여 불러올 수 있습니다.

```python
causal_lm = keras_nlp.models.CausalLM.from_preset(f"hf://{hf_username}/gpt2_imdb")
```

## 분류기 업로드

분류기 모델 업로드는 Causal LM 업로드와 유사합니다.
미세 조정된 모델을 업로드하려면, 먼저 `save_to_preset` API를 사용하여 모델을 로컬 디렉토리에 저장한 후,
[`keras_nlp.upload_preset`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/base_classes/upload_preset#uploadpreset-function)를 통해 업로드할 수 있습니다.

In [ ]:
# 기본 모델 로드.
classifier = keras_nlp.models.Classifier.from_preset(
    "bert_tiny_en_uncased", num_classes=2
)

# 분류기 미세 조정.
classifier.fit(imdb_train)

# 로컬 프리셋 디렉토리에 모델 저장.
preset_dir = "./bert_tiny_imdb"
classifier.save_to_preset(preset_dir)

# Kaggle에 업로드.
keras_nlp.upload_preset(
    f"kaggle://{kaggle_username}/bert/keras/bert_tiny_imdb", preset_dir
)

모델이 Kaggle에 업로드된 것을 확인한 후, `from_preset`을 호출하여 모델을 불러올 수 있습니다.

```python
classifier = keras_nlp.models.Classifier.from_preset(
    f"kaggle://{kaggle_username}/bert/keras/bert_tiny_imdb"
)
```
